In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X, K):  # @save
    """Compute the 2D cross-correlation"""
    h, w = K.shape
    # The size of the output is gotten by subtracting the size of the Kernel and the size of the input
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i : i + h, j : j + w] * K).sum()
    return Y

In [3]:
# Testing the 2D correlation of the tensor
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])

corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
class Conv2D(nn.Module):
    """A convolution layer applies cross correlation to the input and then adds a scalar bias to the output"""

    def __init__(self, kernel_size) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [5]:
# Understanding Edge detection

X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
K = torch.tensor([[1.0, -1.0]])  # The kernel is height 1, width 2 [1, 2] matrix
# This kernel only detects vertical edges. Calculates X[i, j] - X[i+1, i]
Y = corr2d(X, K)
# notice the edge detection. 1 => D
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [16]:
# Now we want to learn the kernel which transforms X => Y i.e The ernel which does the vertical edge detection without guessing it as above
# The real kernel is [[1,2]] 2nd orther tensor

# Construct a 1 output channel Convolutional Network with a kernel of shape (1,2) and no bias
# initialize kernel to random values
conv = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The input is a 4th order tensor (example, channel, height, width) since we are woring with batch data
# e.g X is reshaped as: 1 example, 1 channel, 6(height) by 6(width) picture

X = X.reshape((1, 1, 6, 8))  # Input
Y = Y.reshape((1, 1, 6, 7))  # Expected Output
lr = 3e-2

for i in range(20):
    Y_hat = conv(X)
    l = (Y_hat - Y) ** 2
    conv.zero_grad()
    l.sum().backward()

    # Update weights or kernel
    conv.weight.data[:] -= lr * conv.weight.grad
    if (i + 1) % 2 == 0:
        print(f"epoch {i + 1}, loss {l.sum():.3f}")
    # if (i+1)@
conv.weight.data.reshape((1,2)) # Answer is approximately [1,-1]